In [7]:
# Install the following libraries
# %pip install sepaxml

In [8]:
# Import libraries
import json
import sys
sys.path.append("..")
from utils.filesystem import *

# Import SEPA classes
from sepaxml import SepaDD

In [9]:
# Config file path
config_file = '../config.json'

# Load the config file
with open(config_file) as f:
    config = json.load(f)

# Load the basic config parameters
input_folder_remesa = config['input_folder_remesa']
input_file_remesa = config['input_file_remesa']

In [10]:
# Ignore UserWarning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load a specific worksheet from the Excel file
try:
    dataframe = load_csv_to_dataframe (input_folder_remesa, input_file_remesa, 0)

    # Count the number of rows and columns
    rows, columns = dataframe.shape
    print(f"The dataframe has {rows} rows and {columns} columns")
except Exception as e:
    print(f"An error occurred: {e}")

The dataframe has 896 rows and 12 columns


In [11]:
# Remove rows that have ContactType__c = "Baja"
try:
    dataframe = dataframe[dataframe['ContactType__c'] != 'Baja']
    rows, columns = dataframe.shape
    print(f"The dataframe has {rows} rows and {columns} columns")
except Exception as e:
    print(f"An error occurred: {e}")

# Print the column names
print(dataframe.columns)

The dataframe has 660 rows and 12 columns
Index(['attributes', 'Id', 'ContactType__c', 'FirstName', 'MiddleName',
       'LastName', 'Name', 'Email', 'aesr_CuentaBancaria_NombreTitular__c',
       'aesr_CuentaBancaria_NombreBanco__c', 'aesr_CuentaBancaria_IBAN__c',
       'aesrCuotaSocio__c'],
      dtype='object')


In [16]:
import pandas as pd
from sepaxml import SepaDD
import datetime

# Sample dataframe similar to what you have
data = {
    'attributes': [''] * 3,
    'Id': ['001', '002', '003'],
    'ContactType__c': ['Individual', 'Individual', 'Individual'],
    'FirstName': ['John', 'Jane', 'Doe'],
    'MiddleName': ['', '', ''],
    'LastName': ['Doe', 'Smith', 'Johnson'],
    'Name': ['John Doe', 'Jane Smith', 'Doe Johnson'],
    'Email': ['john.doe@example.com', 'jane.smith@example.com', 'doe.johnson@example.com'],
    'aesr_CuentaBancaria_NombreTitular__c': ['John Doe', 'Jane Smith', 'Doe Johnson'],
    'aesr_CuentaBancaria_NombreBanco__c': ['Bank A', 'Bank B', 'Bank C'],
    'aesr_CuentaBancaria_IBAN__c': ['DE89370400440532013000', 'FR7630006000011234567890189', 'ES9121000418450200051332'],
    'aesrCuotaSocio__c': [100.0, 200.0, 150.0]
}

df = pd.DataFrame(data)

# Configuration for the SEPA Direct Debit
config = {
    "name": "Your Company Name",
    "IBAN": "YOUR_IBAN_HERE",
    "BIC": "YOUR_BIC_HERE",
    "batch": True,
    "creditor_id": "YOUR_CREDITOR_ID_HERE",  # supplied by your bank or financial authority
    "currency": "EUR",  # ISO 4217
    "address": {
        "address_type": "ADDR",
        "department": "Head Office",
        "subdepartment": None,
        "street_name": "Musterstr.",
        "building_number": "1",
        "postcode": "12345",
        "town": "Berlin",
        "country": "DE",
        "country_subdivision": None,
        "lines": ["Line 1", "Line 2"],
    },
}

# Initialize the SEPA Direct Debit
sepa = SepaDD(config, schema="pain.008.001.02", clean=True)

# Adding transactions from the dataframe
for _, row in df.iterrows():
    payment = {
        "name": row['aesr_CuentaBancaria_NombreTitular__c'],
        "IBAN": row['aesr_CuentaBancaria_IBAN__c'],
        "BIC": "",  # Optional: Add BIC if available
        "amount": int(row['aesrCuotaSocio__c'] * 100),  # Convert to cents
        "type": "OOFF",  # One-Off Direct Debit
        "collection_date": datetime.date.today(),
        "mandate_id": 'MANDATE_' + row['Id'],
        "mandate_date": datetime.date.today(),
        "description": 'Membership Fee',
    }
    sepa.add_payment(payment)

# Export the XML file without validation
xml_output = sepa.export(validate=False)
print(xml_output.decode('utf-8'))

# Save the XML to a file for further inspection
with open("sepa_dd_unvalidated.xml", "wb") as f:
    f.write(xml_output)


<?xml version="1.0" encoding="UTF-8"?><Document xmlns="urn:iso:std:iso:20022:tech:xsd:pain.008.001.02" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><CstmrDrctDbtInitn><GrpHdr><MsgId>20240524055943-f355f4e06b65</MsgId><CreDtTm>2024-05-24T17:59:43</CreDtTm><NbOfTxs>3</NbOfTxs><CtrlSum>450.00</CtrlSum><InitgPty><Nm>Your Company Name</Nm><Id><OrgId><Othr><Id>YOUR_CREDITOR_ID_HERE</Id></Othr></OrgId></Id></InitgPty></GrpHdr><PmtInf><PmtInfId>YourCompanyName-685c44daa7a7</PmtInfId><PmtMtd>DD</PmtMtd><BtchBookg>true</BtchBookg><NbOfTxs>3</NbOfTxs><CtrlSum>450.00</CtrlSum><PmtTpInf><SvcLvl><Cd>SEPA</Cd></SvcLvl><LclInstrm><Cd>CORE</Cd></LclInstrm><SeqTp>OOFF</SeqTp></PmtTpInf><ReqdColltnDt>2024-05-24</ReqdColltnDt><Cdtr><Nm>Your Company Name</Nm><PstlAdr><AdrTp>ADDR</AdrTp><Dept>Head Office</Dept><StrtNm>Musterstr.</StrtNm><BldgNb>1</BldgNb><PstCd>12345</PstCd><TwnNm>Berlin</TwnNm><Ctry>DE</Ctry><AdrLine>Line 1</AdrLine><AdrLine>Line 2</AdrLine></PstlAdr></Cdtr><CdtrAcct><Id><IBAN>YOU